In [1]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.feature_extraction.text import CountVectorizer
import re, codecs
from bs4 import BeautifulSoup

In [ ]:
def re_encode(source_file,
            source_encoding,
            destination_file,
            destination_encoding):
    """
    Change encoding of a file and write it as a new file
    """
    with codecs.open(source_file,
                    'r',
                    source_encoding) as source_handle:
        with codecs.open(destination_file,
                        'w',
                        destination_encoding) as destination_handle:
            contents = source_handle.read()
            destination_handle.write(contents)

In [2]:
def standardize_column_names(df):
    """
    Convert all columns to lowercase
    Replace spaces with underscores
    """
    prior_columns = list(df.columns)
    new_columns = [column_name.lower().replace(' ', '_') for column_name in prior_columns]
    df.columns = new_columns

In [3]:
def clean_html_body(text_body):
    """
    Pseudocode

    Remove <pre> tag section
    Remove <code> tag section
    Remove <anchor> tag section
    Extract all text from the html content
    Remove new line characters
    Return clean text
    """
    pre_tag_pattern = r'<pre>[\s|\S|\d|\D|\w|\W]*?</pre>'
    code_tag_pattern = r'<code>[\s|\S|\d|\D|\w|\W]*?</code>'
    anchor_tag_pattern = r'<a[\s|\S|\d|\D|\w|\W]*?</a>'
    pre_tag_removed = re.sub(pre_tag_pattern, 'lala',  text_body, flags=re.IGNORECASE)
    code_tag_removed = re.sub(code_tag_pattern, ' ', pre_tag_removed, flags=re.IGNORECASE)
    anchor_tag_removed = re.sub(anchor_tag_pattern, ' ', code_tag_removed, flags=re.IGNORECASE)
    return BeautifulSoup(anchor_tag_removed).get_text().encode('utf-8').decode('utf-8')

In [ ]:

# Re encoding files to utf-8 format            
            
re_encode('Data/Questions.csv',
         'macintosh',
         'Data/Questions_utf8.csv',
         'utf-8')       

re_encode('Data/Answers.csv',
         'macintosh',
         'Data/Answers_utf8.csv',
         'utf-8')  

re_encode('Data/Tags.csv',
         'macintosh',
         'Data/Tags_utf8.csv',
         'utf-8')  

In [4]:
questions = pd.read_csv('Data/Questions_utf8.csv',
                       encoding='utf-8')
answers = pd.read_csv('Data/Answers_utf8.csv')
tags = pd.read_csv('Data/Tags_utf8.csv')

# Standardizing columns
standardize_column_names(questions)
standardize_column_names(answers)
standardize_column_names(tags)

In [ ]:
# In[6]:


questions.head(5)

In [ ]:
# In[7]:


answers.head(5)

In [ ]:
# In[8]:


tags.head(5)

In [5]:
# ### Restructuring `tags` data frame

# In[6]:


num_missing_tags = tags.tag.isnull().sum()
print(f'Number of questions missing tags: {num_missing_tags}')

Number of questions missing tags: 443


In [6]:
# *Removing these rows from the data frame*

# In[7]:


tags.dropna(inplace=True)

In [7]:
# ### Grouping tags based on question id

# In[8]:


tags = tags[["id", "tag"]].groupby('id')['tag'].apply(' '.join).reset_index()
tags.head()

,id,tag
0,469,python osx fonts photoshop
1,502,python windows image pdf
2,535,python continuous-integration extreme-programming
3,594,python sql database oracle cx-oracle
4,683,python arrays iteration


In [8]:
# ### Merging `tags` df with `questions` df

# In[9]:


df = pd.merge(left=questions,
                     right=tags,
                     how='left',
                     on='id')
df.rename(columns={'tag': 'tags'},
         inplace=True)
df.head()

,id,owneruserid,creationdate,score,title,body,tags
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,python osx fonts photoshop
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,python windows image pdf
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,python continuous-integration extreme-programming
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,python sql database oracle cx-oracle
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,python arrays iteration


In [9]:
# ### Merging `answers` with `questions`

# In[10]:


temp_ans = answers[['parentid', 'body']].copy()
temp_ans_rename = {
    'parentid': 'id',
    'body': 'answer_body'
}
temp_ans.rename(columns=temp_ans_rename,
                          inplace=True)

temp_ans = temp_ans[['id', 'answer_body']].groupby('id')['answer_body'].apply(' '.join).reset_index()
raw_df = pd.merge(left=df,
                  right=temp_ans,
                  on='id',
                  how='left')
raw_df.head()

,id,owneruserid,creationdate,score,title,body,tags,answer_body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,python osx fonts photoshop,<p>open up a terminal (Applications-&gt;Utilit...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,python windows image pdf,<p>You can use ImageMagick's convert utility f...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,python continuous-integration extreme-programming,<p>One possibility is Hudson. It's written in...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,python sql database oracle cx-oracle,<p>The canonical way is to use the built-in cu...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,python arrays iteration,"<p>No, you were not dreaming. Python has a pr..."


In [46]:
# Checking for encoding errors 

remove_indices = list()
for text_ in raw_df[['body']].iterrows():
    index = text_[0]
    q = text_[1][0]
    try:
        x = clean_html_body(q)
    except:
        print(index)
        remove_indices.append(index)
print(remove_indices)

33800
[33800]


In [54]:
### Extract text from question body after removing row at index 33800
raw_df.drop([33800],
           inplace=True)

In [55]:
raw_df.to_csv('Data/raw_df.csv',
             index=False)

In [56]:
raw_df['clean_q_body'] = raw_df['body'].apply(lambda x: clean_html_body(x))
raw_df['all_text'] = raw_df[['clean_q_body', 'tags', 'title']].apply(lambda x: ' '.join(x), axis=1)

In [57]:
raw_df.to_csv('Data/raw_df_2.csv',
             index=False)